# Registration

Updated 22-02-2023

Experimenting with transforming the mask and performing multiple transformations subsequently

For now I only experimented with transforming one moving / atlas image (p102) towards one fixed / validation image (p135)

## Importing and displaying the data

The idea is to use 12 patients as atlases (p102 to p128), 3 patients for validation 129, 133, 135.
The atlases are the moving images that we have to transform to align with the fixed image (validation images).

Hence below I will do a first attempt at registering p102 as moving image to p135 as fixed image with a rigid transformation using Elastix.

If I have acquired the transformation for p102 I will also apply the same transformation to the mask / segmentation of p102 and compare it to the actual mask of p135 to see how well the registration method performs.

In [2]:
# Import packages
%matplotlib qt 
import SimpleITK as sitk
from scrollview import ScrollView
import numpy as np
import matplotlib.pyplot as plt
from IndexTracker import IndexTracker
import imageio
import elastix
import os

In [3]:
# Create paths to elastix packages

ELASTIX_PATH = 'C:\\Users\\20192024\\OneDrive - TU Eindhoven\\Documents\\Y4\\Q3\\Capita Selecta in Medical Image Analysis\\myfolder\\elastix.exe'
TRANSFORMIX_PATH = 'C:\\Users\\20192024\\OneDrive - TU Eindhoven\\Documents\\Y4\\Q3\\Capita Selecta in Medical Image Analysis\\myfolder\\transformix.exe'
project_dir = '..\..'

parameter_files = '..\Registration'

if not os.path.exists(ELASTIX_PATH):
    raise IOError('Elastix cannot be found, please set the correct ELASTIX_PATH.')

if not os.path.exists(TRANSFORMIX_PATH):
    raise IOError('Transformix cannot be found, please set the correct TRANSFORMIX_PATH.')
    

# Make a results directory if non exists
if os.path.exists(os.path.join(project_dir, 'results')) is False:
    os.mkdir(os.path.join(project_dir, 'results'))

### Atlas patient p102


In [4]:
# Define paths and load data
data = '..\..\TrainingData'
patients = os.listdir(data)

# Import the image and its segmentation for the atlas patient 102
# The following path will change if we loop through different patients (loop needs to be inserted here)
at_patient = 'p102'
atlas_path = os.path.join(data, at_patient)

In [5]:
atlas_path

'..\\..\\TrainingData\\p102'

In [4]:
# Load image and segmentation of atlas patient / patients
atlas_im_path = os.path.join(atlas_path, 'mr_bffe.mhd')
itk_image102 = sitk.ReadImage(atlas_im_path)
image_array102 = sitk.GetArrayViewFromImage(itk_image102)

# Print the image dimensions
print(image_array102.shape)

atlas_seg_path = os.path.join(atlas_path, 'prostaat.mhd')
itk_image_seg = sitk.ReadImage(atlas_seg_path)
segmentation102 = sitk.GetArrayViewFromImage(itk_image_seg)

# Print the image dimensions
print(segmentation102.shape)

(86, 333, 271)
(86, 333, 271)


In [ ]:
# Show the image
fig, ax = plt.subplots(1, 2)
tracker1 = IndexTracker(ax[0], image_array102);
tracker2 = IndexTracker(ax[1], segmentation102);
fig.canvas.mpl_connect('scroll_event', tracker1.onscroll);
fig.canvas.mpl_connect('scroll_event', tracker2.onscroll);

plt.show()

### Validation patient p135

In [5]:
# Import the image and its segmentation for the atlas patient
val_patient = 'p135'
validation_path = os.path.join(data, val_patient)

validation_im_path = os.path.join(validation_path, 'mr_bffe.mhd')
itk_image135 = sitk.ReadImage(validation_im_path)
image_array135 = sitk.GetArrayViewFromImage(itk_image135)


validation_seg_path = os.path.join(validation_path, 'prostaat.mhd')
itk_image_seg = sitk.ReadImage(validation_seg_path)
segmentation135 = sitk.GetArrayViewFromImage(itk_image_seg)

In [ ]:
# Show the image
fig, ax = plt.subplots(1, 2)
tracker1 = IndexTracker(ax[0], image_array135);
tracker2 = IndexTracker(ax[1], segmentation135);
fig.canvas.mpl_connect('scroll_event', tracker1.onscroll);
fig.canvas.mpl_connect('scroll_event', tracker2.onscroll);


plt.show()

## Perform the registration

In the manual (p. 27) they say: Always start with a rigid or affine transformation before doing a nonrigid one, to get a good initial alignment.

### Translation & B-spline 64 registration with Par0001 parameter file (from ElastixZoo)

In [6]:
# Make atlas specific result directory
if os.path.exists(os.path.join(project_dir, "results", at_patient)) is False:
    os.mkdir(os.path.join(project_dir, "results", at_patient))
res_path = os.path.join(project_dir, "results", at_patient)

# Define a new elastix object 'el' with the correct path to elastix
el = elastix.ElastixInterface(elastix_path=ELASTIX_PATH)

moving = os.path.join(atlas_path, 'mr_bffe.mhd')
fixed = os.path.join(validation_path, 'mr_bffe.mhd')

moving_seg = os.path.join(atlas_path, 'prostaat.mhd')
fixed_seg = os.path.join(validation_path, 'prostaat.mhd')

parameter0 = os.path.join(parameter_files, 'Par0001translation.txt')
parameter1 = os.path.join(parameter_files, 'Par0001bspline64.txt')
# Register the two images

el.register(
    fixed_image=fixed,
    moving_image=moving,
    parameters = [parameter0, parameter1],
    output_dir=res_path);

if os.path.exists(os.path.join(res_path, "mr_bffe")) is False:
    os.mkdir(os.path.join(res_path, "mr_bffe"))

if os.path.exists(os.path.join(res_path, "prostaat")) is False:
    os.mkdir(os.path.join(res_path, "prostaat"))


C:\Users\20192024\OneDrive - TU Eindhoven\Documents\Y4\Q3\Capita Selecta in Medical Image Analysis\myfolder\elastix.exe -f ..\..\TrainingData\p135\mr_bffe.mhd -m ..\..\TrainingData\p102\mr_bffe.mhd -p ..\Registration\Par0001translation.txt -p ..\Registration\Par0001bspline64.txt -out ..\..\results\p102


### Solution to ripple problem

[Here](https://github.com/SuperElastix/SimpleElastix/issues/251) they mention that in order to transform a binary mask, the parameter "FinalBSplineInterpolationOrder" must be set to 0 in transform parameter files (I think in the transform parameter files, because in the normal parameter files this is not necessary as we only use those for the images and not for the binary masks). Hence I have created two copies of the transform parameter files for translation and b spline ("TransformParameters.1.binary.txt") where I changed that parameter FinalBSplineInterpolationOrder from 3 to 0 and will thus do a separate transformation on the masks with those specific parameter files.

Also, regarding combining transformations, I think the last transformparameters.txt (TransformParameters.1.txt) file has already incorporated the first transformation because when only using TransformParameters.1.txt the binary is mask is translated as well as deformed + in the Transform parameter files there is an option on how to combine transformations. The options are 'Add' (combine transformations by adding them) or 'Compose' (default, by composition).

In [7]:
# Load transformation

trans_parameters1 = os.path.join(res_path, "TransformParameters.1.txt")
trans_parameters1_seg = os.path.join(res_path, "TransformParameters.1.binary.txt")

# First transformation (= translation)

tr = elastix.TransformixInterface(parameters=trans_parameters1, transformix_path=TRANSFORMIX_PATH)
tr_seg = elastix.TransformixInterface(parameters=trans_parameters1_seg, transformix_path=TRANSFORMIX_PATH)

tr.transform_image(moving, output_dir=os.path.join(res_path, "mr_bffe"))
tr_seg.transform_image(moving_seg, output_dir=os.path.join(res_path, "prostaat"))

# Intermediary result
transformed_moving_image_0 = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(res_path, "mr_bffe", "result.mhd")))
transformed_moving_label_0 = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(res_path, "prostaat", "result.mhd")))


In [13]:
transformed_moving_label_0.shape

(86, 333, 271)

In [14]:
from scipy.spatial.distance import directed_hausdorff

slices = transformed_moving_label_0.shape[0]


directed_hausdorff(segmentation135[40], transformed_moving_label_0[40])[0]

2.6457513110645907

In [ ]:
# Show transformed images
fig, ax = plt.subplots(1, 4)
tracker1 = IndexTracker(ax[0], image_array135);
tracker2 = IndexTracker(ax[1], segmentation135);
tracker3 = IndexTracker(ax[2], transformed_moving_image_0);
tracker4 = IndexTracker(ax[3], transformed_moving_label_0);
fig.canvas.mpl_connect('scroll_event', tracker1.onscroll);
fig.canvas.mpl_connect('scroll_event', tracker2.onscroll);
fig.canvas.mpl_connect('scroll_event', tracker3.onscroll);
fig.canvas.mpl_connect('scroll_event', tracker4.onscroll);

ax[0].set_title("Fixed image")
ax[1].set_title("Fixed image segmentation")
ax[2].set_title("Transformed moving image")
ax[3].set_title("Transformed moving seg")

plt.show()

### Previous stuff I tried (13-2-2023)

In [ ]:


# Jacobian
tr = elastix.TransformixInterface(parameters='results/TransformParameters.0.txt', transformix_path=TRANSFORMIX_PATH) 
jacobian_matrix_path = tr.jacobian_matrix(output_dir=r'results')
jacobian_determinant_path = tr.jacobian_determinant(output_dir=r'results')

jacobian_path = 'results/spatialJacobian.mhd'

itk_image_jac = sitk.ReadImage(jacobian_path)
image_array_jac =sitk.GetArrayFromImage(itk_image_jac)
out = image_array_jac>0

# Visualize results
fig, ax = plt.subplots(1, 2)
tracker1 = IndexTracker(ax[0], image_array);
tracker2 = IndexTracker(ax[1], out);
fig.canvas.mpl_connect('scroll_event', tracker1.onscroll);
fig.canvas.mpl_connect('scroll_event', tracker2.onscroll);

ax[0].set_title("B-spline registration, no penalty term")
ax[1].set_title("Corresponding Jacobian")

We see in the plot of the Jacobian that we have a lot of yellow, hence where the jacobian is negative / negative values and thus where we have folding. So this transformation on itself is not successful. 

### B-spline registration no multi-grid and with penalty

In [ ]:
# Define a new elastix object 'el' with the correct path to elastix

el = elastix.ElastixInterface(elastix_path=ELASTIX_PATH)


# Register the two images
el.register(
    fixed_image='../../TrainingData/p127/mr_bffe.mhd',
    moving_image='../../TrainingData/p102/mr_bffe.mhd',
    parameters = [r"C:\\Users\\20192024\\OneDrive - TU Eindhoven\\Documents\\Y4\\Q3\\Capita Selecta in Medical Image Analysis\\Project\\CapitaSelectaCode\\opendata\\parameter_files\\parameters_bspline_penalty.txt"],
    output_dir="results");

# Find the results
transform_path = os.path.join('results', 'TransformParameters.0.txt');
result_path = os.path.join('results', 'result.0.mhd');



In [ ]:
# Load results
# Resulting transformed image
itk_image = sitk.ReadImage(result_path)
image_array =sitk.GetArrayFromImage(itk_image)

# Jacobian
tr = elastix.TransformixInterface(parameters='results/TransformParameters.0.txt', transformix_path=TRANSFORMIX_PATH) 
jacobian_matrix_path = tr.jacobian_matrix(output_dir=r'results')
jacobian_determinant_path = tr.jacobian_determinant(output_dir=r'results')

jacobian_path = 'results/spatialJacobian.mhd'

itk_image_jac = sitk.ReadImage(jacobian_path)
image_array_jac =sitk.GetArrayFromImage(itk_image_jac)
out = image_array_jac>0

# Visualize results
fig, ax = plt.subplots(1, 2)
tracker1 = IndexTracker(ax[0], image_array);
tracker2 = IndexTracker(ax[1], out);
fig.canvas.mpl_connect('scroll_event', tracker1.onscroll);
fig.canvas.mpl_connect('scroll_event', tracker2.onscroll);

ax[0].set_title("B-spline registration, with penalty term")
ax[1].set_title("Corresponding Jacobian")

Plat transformed moving image next to fixed image

In [ ]:
# Visualize results
fig, ax = plt.subplots(1, 2)
tracker1 = IndexTracker(ax[0], image_array);
tracker2 = IndexTracker(ax[1], image_array127);
fig.canvas.mpl_connect('scroll_event', tracker1.onscroll);
fig.canvas.mpl_connect('scroll_event', tracker2.onscroll);

ax[0].set_title("Transformed moving image with Bspline and penalty (p102)")
ax[1].set_title("Fixed image (p127)")




## Affine registration followed by B-spline (with penalty, multi-resolution, no multi-grid)

They mention in the manual that using multi-grid might improve our results but I still have to experiment with that

### Rigid registration (aborted for now)

In [ ]:
fixed_image = image_array127
moving_image = image_array102

# Import Default Parameter Map
parameter_object = itk.ParameterObject.New()
default_rigid_parameter_map = parameter_object.GetDefaultParameterMap('rigid')
parameter_object.AddParameterMap(default_rigid_parameter_map)

In [ ]:
# Call registration function
result_image, result_transform_parameters = itk.elastix_registration_method(
    fixed_image, moving_image,
    parameter_object=parameter_object,
    log_to_console=True)

In [ ]:
%matplotlib inline

# Plot images
fig, axs = plt.subplots(1,3, sharey=True, figsize=[30,30])
plt.figsize=[100,100]
axs[0].imshow(result_image)
axs[0].set_title('Result', fontsize=30)
axs[1].imshow(fixed_image)
axs[1].set_title('Fixed', fontsize=30)
axs[2].imshow(moving_image)
axs[2].set_title('Moving', fontsize=30)
plt.show()